# Simple Keras Layer Demo

This notebook demonstrates how to use the `KerasCircuitLayer` from `pennylane-keras-layer` to create a hybrid quantum-classical model using Keras 3.

## 0. Setup and Installation

First, execute the installation commands if you haven't already.

In [1]:
# !pip install tensorflow  # Or jax, torch
# !pip install pennylane
# !pip install pennylane-keras-layer

In [2]:
import os

# Set backend to JAX (optional, but good for performance)
os.environ["KERAS_BACKEND"] = "jax"

import keras
import pennylane as qml
import numpy as np

# Import the layer
from pennylane_keras_layer import KerasCircuitLayer

## 1. Create a Random QNode

We define a simple QNode that uses `AngleEmbedding` for inputs and `StronglyEntanglingLayers` for variational weights.

In [3]:
n_qubits = 2
n_layers = 2
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def qnode(weights, inputs):
    """
    A simple QNode that embeds input data and then applies layers of weights.
    Structure:
    - AngleEmbedding for inputs
    - StronglyEntanglingLayers for weights
    """
    qml.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

# Define weight shapes for the Keras layer
# StronglyEntanglingLayers expects shape (n_layers, n_qubits, 3)
weight_shapes = {"weights": (n_layers, n_qubits, 3)}
print(f"QNode defined with {n_qubits} qubits and {n_layers} layers.")

QNode defined with 2 qubits and 2 layers.


## 2. Wrap it into Keras

Now we wrap the PennyLane QNode into a Keras layer using `KerasCircuitLayer`. We specify the `output_dim` to match the number of measurements.

In [4]:
# output_dim matches the number of measurements (2 expvals)
qlayer = KerasCircuitLayer(qnode, weight_shapes, output_dim=n_qubits)
print("KerasCircuitLayer created.")

KerasCircuitLayer created.


## 3. Create a Model and Train

We build a simple regression model: `Input -> Quantum Layer -> Dense Output`. We generate some random dummy data and train the model.

In [5]:
# Simple model: Input -> Quantum Layer -> Output (Dense)
inputs = keras.Input(shape=(n_qubits,))
x = qlayer(inputs)
outputs = keras.layers.Dense(1)(x) # Regression output

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer="adam", loss="mse")

# Generate some random dummy data
# batch_size=10, features=2
X = np.random.random((10, n_qubits))
y = np.random.random((10, 1))

print("Starting training...")
history = model.fit(X, y, epochs=5, batch_size=2, verbose=1)
print("Training complete.")

Starting training...
Epoch 1/5


1/5 ━━━━━━━━━━━━━━━━━━━━ 1s 295ms/step - loss: 1.3683

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.1112  


Epoch 2/5


1/5 ━━━━━━━━━━━━━━━━━━━━ 0s 637us/step - loss: 1.2635

...

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 165us/step - loss: 0.9785


Epoch 5/5


1/5 ━━━━━━━━━━━━━━━━━━━━ 0s 416us/step - loss: 0.8811

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 148us/step - loss: 0.9396


Training complete.


## 4. Saving and Loading

We save the trained model to a `.keras` file and reload it. 

> **Important:** Since QNodes are not natively serializable, we *must* manually restore the QNode using the `set_qnode()` method after loading the layer.

In [6]:
save_path = "simple_qnn.keras"
model.save(save_path)
print(f"Model saved to {save_path}")

Model saved to simple_qnn.keras


In [7]:
# Load the model
loaded_model = keras.models.load_model(save_path)
print("Model loaded successfully.")

# Verify QNode restoration
print("Restoring QNode to loaded layer...")

# Iterate to find the KerasCircuitLayer
q_layer_loaded = None
for layer in loaded_model.layers:
    if isinstance(layer, KerasCircuitLayer):
        q_layer_loaded = layer
        break

if q_layer_loaded:
    q_layer_loaded.set_qnode(qnode)
    print("QNode set on loaded layer.")
else:
    print("Warning: Could not find KerasCircuitLayer in loaded model.")

Delaying circuit creation till QNode is set using the 'set_qnode' method
Model loaded successfully.
Restoring QNode to loaded layer...
Verifying QNode compatibility
Updating QNode
QNode set on loaded layer.


## 5. Verify Inference

Finally, we verify that the loaded model can perform inference.

In [8]:
print("Verifying inference with loaded model...")
try:
    pred = loaded_model.predict(X[:2])
    print(f"Prediction shape: {pred.shape}")
    print("Inference successful!")
except Exception as e:
    print(f"Inference failed: {e}")

Verifying inference with loaded model...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step


Prediction shape: (2, 1)
Inference successful!


In [9]:
# Cleanup
if os.path.exists(save_path):
    os.remove(save_path)
    print(f"Cleaned up {save_path}")

Cleaned up simple_qnn.keras
